In [ ]:
'''
Figure of the BIC for the full cube models.
'''

%matplotlib inline

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.convolution import Gaussian1DKernel

import os
import numpy as np
import matplotlib.pyplot as plt
from spectral_cube import Projection
from scipy import ndimage as nd
from astropy.stats import histogram as astro_hist
from tqdm import tqdm
from corner import hist2d

from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

osjoin = os.path.join

repo_path = os.path.expanduser("~/ownCloud/project_code/ThickHIFitting/")

figures_path_png = osjoin(repo_path, "figures/png")
figures_path_pdf = osjoin(repo_path, "figures/pdf")


def save_figure(fig, plot_name, **kwargs):
    fig.savefig(f"{figures_path_pdf}/{plot_name}.pdf", **kwargs)
    fig.savefig(f"{figures_path_png}/{plot_name}.png", **kwargs)


paths_script = os.path.join(repo_path, "paths.py")
exec(compile(open(paths_script, "rb").read(), paths_script, 'exec'))

plotstyle_script = os.path.join(repo_path, "plotting_styles.py")
exec(compile(open(plotstyle_script, "rb").read(), plotstyle_script, 'exec'))

model_script = os.path.join(repo_path, "gaussian_model.py")
exec(compile(open(model_script, "rb").read(), model_script, 'exec'))

thickHI_model_script = os.path.join(repo_path, "thickHI_model.py")
exec(compile(open(thickHI_model_script, "rb").read(),
             thickHI_model_script, 'exec'))


In [ ]:
# Enable/disable saving figures
save_figures = True

In [ ]:
# M31

m31_cubename_K = f"{fifteenA_HI_BCtaper_wEBHIS_HI_file_dict['Cube'].rstrip('.fits')}_K.fits"

m31_cube = SpectralCube.read(m31_cubename_K, use_dask=True)
print(f'Opening cube {m31_cubename_K}')

m31_vels = m31_cube.spectral_axis.to(u.m / u.s)
      
# del m31_cube

m31_mom0 = Projection.from_hdu(fits.open(fifteenA_HI_BCtaper_wEBHIS_HI_file_dict['Moment0'])).to(u.K * u.km / u.s)
      
m31_multigauss_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_multigaussian_gausspy_fits_neighbcheck2_nomw.fits")
# m31_multigauss_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_multigaussian_gausspy_fits_neighbcheck2.fits")
m31_multigauss_hdu = fits.open(m31_multigauss_name)

m31_ngauss = np.isfinite(m31_multigauss_hdu[0].data).sum(0) // 3

m31_thickHI_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_simplethick_HI_fits_5kms_centlimit.fits")
m31_thickHI_hdu = fits.open(m31_thickHI_name)

m31_thickHI80_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_simplethick_HI_fits_80kms_centlimit.fits")
m31_thickHI80_hdu = fits.open(m31_thickHI_name)

# Keep only where the fit parameters are valid
m31_multigauss_hdu[2].data[m31_ngauss == 0] = np.NaN

m31_multigauss_bic_proj = Projection.from_hdu(m31_multigauss_hdu[2])
      
# Split the different fit statistics
m31_thickHI_bic_proj = Projection.from_hdu(fits.PrimaryHDU(m31_thickHI_hdu[2].data[0], m31_thickHI_hdu[2].header))
m31_thickHI80_bic_proj = Projection.from_hdu(fits.PrimaryHDU(m31_thickHI80_hdu[2].data[0], m31_thickHI_hdu[2].header))
      
m31_thickHI_rchi_proj = Projection.from_hdu(fits.PrimaryHDU(m31_thickHI_hdu[2].data[2], m31_thickHI_hdu[2].header))
m31_thickHI80_rchi_proj = Projection.from_hdu(fits.PrimaryHDU(m31_thickHI80_hdu[2].data[2], m31_thickHI_hdu[2].header))
      
# Lastly, the recomputed statistics limited to where tau > 0.5
m31_modcompare_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_recomp_bic_tau_gt_0p5_5kms_centlimit.fits")
m31_modcompare_hdu = fits.open(m31_modcompare_name)
      
m31_modcompare80_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_recomp_bic_tau_gt_0p5_80kms_centlimit.fits")
m31_modcompare80_hdu = fits.open(m31_modcompare_name)
      
# Slice out to zoom into the valid data region.
spat_slice_zoom_m31 = tuple([slice(vals.min() - 10, vals.max() + 10) for vals in
                             np.where(np.isfinite(m31_multigauss_hdu[2].data))])
print(spat_slice_zoom_m31)
# Make custom slice so the array shapes are roughly the same in M33 and M31
spat_slice_zoom_m31 = (slice(105, 1709), slice(347, 1603))

In [ ]:
m31_cube

In [ ]:
%matplotlib inline

plt.imshow(m31_mom0.value, origin='lower')

In [ ]:
region_slice = (slice(850, 1050), slice(900, 1100))

plt.imshow(m31_mom0[region_slice].value, origin='lower')


We're going use this $200^2$ pixels to spectrally degrade the data. Also we'll convolve to a lower resolution.

In [ ]:
# Load in the already made subcube from degraded_resolution_test_fits.py

subcube_filename = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("match_braun09/M31_HI_subcube_K_30arcsec_2p4kms.fits", no_check=True)

matched_cube = SpectralCube.read(subcube_filename)

In [ ]:
matched_cube

In [ ]:
# Load in the subcube fit results

hdu_thickHI_matched = fits.open(fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("braun09_subcubes/matched_region_opaque_fit_comparisons.fits"))

hdu_mgauss_matched = fits.open(fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("braun09_subcubes/matched_region_multigauss_fit_comparisons.fits"))

In [ ]:
orig_avgspec = m31_cube[(slice(None),) + region_slice].mean(axis=(1, 2))
matched_avgspec = matched_cube.mean(axis=(1, 2))

In [ ]:
# We'll now plot the Delta BIC as histograms between the original and the matched fits.
# Show an example spectrum at high and low spec. res.

onecolumn_twopanel_figure()

fig, axs = plt.subplots(2, 1)

# axs[0].plot(orig_avgspec.spectral_axis.to(u.km / u.s).value, orig_avgspec.value, drawstyle='steps-mid')
# axs[0].plot(matched_avgspec.spectral_axis.to(u.km / u.s).value, matched_avgspec.value, drawstyle='steps-mid')

ypix, xpix = 60, 195

axs[0].plot(orig_avgspec.spectral_axis.to(u.km / u.s).value,
            m31_cube[(slice(None),) + region_slice][:, ypix, xpix].value,
            drawstyle='steps-mid')
axs[0].plot(matched_avgspec.spectral_axis.to(u.km / u.s).value,
            matched_cube[:, ypix, xpix].value,
            drawstyle='steps-mid', linewidth=2)

axs[0].set_xlim([-200, -80])

axs[0].set_xlabel("Velocity (km s$^{-1}$)")
axs[0].set_ylabel("Brightness Temperature (K)")

axs[0].grid(True)

deltaBIC_orig = m31_multigauss_bic_proj[region_slice].value - m31_thickHI_bic_proj[region_slice].value

deltaBIC_matched = hdu_mgauss_matched[2].data[0] - hdu_thickHI_matched[2].data[0]

_ = axs[1].hist(deltaBIC_orig.ravel(), density=True, label=r'0.42 km s$^{-1}$', alpha=0.5)
_ = axs[1].hist(deltaBIC_matched.ravel(), density=True, label=r'2.4 km s$^{-1}$', alpha=0.5)

axs[1].legend(frameon=True, loc='upper left')

axs[1].set_ylabel("Normalized Histogram")
axs[1].set_xlabel(r"$\Delta {\rm BIC} = {\rm BIC}_{\rm Gauss} - {\rm BIC}_{\rm Thick}$")

axs[1].grid(True)

fig.subplots_adjust(hspace=0.25)

if save_figures:
    save_figure(fig, "specres_comparison_deltaBIC_and_spec", bbox_inches='tight', dpi=300)


In [ ]:
from astropy.coordinates import SkyCoord
dec, ra = matched_cube.world[0, 100, 100][1:]

coord = SkyCoord(ra, dec)

In [ ]:
coord.to_string('hmsdms')